# CloudflareWorkersAIEmbeddings

这将帮助您使用LangChain开始使用Cloudflare Workers AI的[嵌入模型](/docs/concepts/embedding_models)。如需关于`CloudflareWorkersAIEmbeddings`功能和配置选项的详细文档，请参阅[API参考文档](https://api.js.langchain.com/classes/langchain_cloudflare.CloudflareWorkersAIEmbeddings.html)。

## 概览
### 集成详情

| 类 | 包 | 本地支持 | Python支持 | 包下载量 | 包最新版本 |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [`CloudflareWorkersAIEmbeddings`](https://api.js.langchain.com/classes/langchain_cloudflare.CloudflareWorkersAIEmbeddings.html) | [`@langchain/cloudflare`](https://npmjs.com/@langchain/cloudflare) | ❌ | ❌ | ![NPM - 下载量](https://img.shields.io/npm/dm/@langchain/cloudflare?style=flat-square&label=%20&) | ![NPM - 版本](https://img.shields.io/npm/v/@langchain/cloudflare?style=flat-square&label=%20&) |

## 安装配置

要访问Cloudflare的嵌入模型，您需要创建一个Cloudflare账户并安装`@langchain/cloudflare`集成包。该集成设计用于在Cloudflare Worker中运行并接受绑定。

请按照[官方文档](https://developers.cloudflare.com/workers-ai/get-started/workers-wrangler/)设置您的Worker。

您的`wrangler.toml`文件应类似于以下内容：

```toml
name = "langchain-test"
main = "worker.js"
compatibility_date = "2024-01-10"

[[vectorize]]
binding = "VECTORIZE_INDEX"
index_name = "langchain-test"

[ai]
binding = "AI"
```

### 凭据

如果您希望对模型调用进行自动追踪，还可以取消以下内容的注释以设置您的[LangSmith](https://docs.smith.langchain.com/) API密钥：

```bash
# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY="your-api-key"
```

### 安装

LangChain CloudflareWorkersAIEmbeddings集成位于`@langchain/cloudflare`包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/cloudflare @langchain/core
</Npm2Yarn>
```

## 使用方法

以下是一个使用 Workers AI 嵌入功能并结合 [Cloudflare Vectorize 向量存储](/docs/integrations/vectorstores/cloudflare_vectorize/) 的示例工作程序。

In [ ]:
// @ts-nocheck

import type {
  VectorizeIndex,
  Fetcher,
  Request,
} from "@cloudflare/workers-types";

import {
  CloudflareVectorizeStore,
  CloudflareWorkersAIEmbeddings,
} from "@langchain/cloudflare";

export interface Env {
  VECTORIZE_INDEX: VectorizeIndex;
  AI: Fetcher;
}

export default {
  async fetch(request: Request, env: Env) {
    const { pathname } = new URL(request.url);
    const embeddings = new CloudflareWorkersAIEmbeddings({
      binding: env.AI,
      model: "@cf/baai/bge-small-en-v1.5",
    });
    const store = new CloudflareVectorizeStore(embeddings, {
      index: env.VECTORIZE_INDEX,
    });
    if (pathname === "/") {
      const results = await store.similaritySearch("hello", 5);
      return Response.json(results);
    } else if (pathname === "/load") {
      // Upsertion by id is supported
      await store.addDocuments(
        [
          {
            pageContent: "hello",
            metadata: {},
          },
          {
            pageContent: "world",
            metadata: {},
          },
          {
            pageContent: "hi",
            metadata: {},
          },
        ],
        { ids: ["id1", "id2", "id3"] }
      );

      return Response.json({ success: true });
    } else if (pathname === "/clear") {
      await store.delete({ ids: ["id1", "id2", "id3"] });
      return Response.json({ success: true });
    }

    return Response.json({ error: "Not Found" }, { status: 404 });
  },
};

## API 参考文档

有关所有 `CloudflareWorkersAIEmbeddings` 功能和配置的详细文档，请访问 API 参考页面: https://api.js.langchain.com/classes/langchain_cloudflare.CloudflareWorkersAIEmbeddings.html